# Top Hostnames
- report externallinks hostname counts for W and WP:M pages
- report various hostname event data for W and WP:M pages

In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

In [2]:
# parse hostnames from links
# includes extraction for web.archive.org links
parse_host = """
    PARSE_URL(REGEXP_REPLACE(LOWER({}),'^https?://web.archive.org/web/[^/]+/',''),'HOST')
"""

## Hostnames present in pages with external links

#### Top 20 hostnames found in WP:M pages with external links

In [3]:
# count of top 20 hostnames in WP:M pages with external links
top_hosts_wpm_query = """
SELECT {} AS host, COUNT(DISTINCT el_from, el_to) AS count 
FROM ryanmax.population_externallinks
WHERE el_from IN 
    (SELECT DISTINCT page_id AS el_from
    FROM ryanmax.population_wpm_pages_with_extlinks
    )
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_wpm = spark.sql(
    top_hosts_wpm_query.format(
        parse_host.format('el_to')
    ))
top_hosts_wpm.toPandas()

,host,count
0,www.ncbi.nlm.nih.gov,238688
1,doi.org,183761
2,www.worldcat.org,25434
3,books.google.com,24491
4,www.google.com,11074
5,apps.who.int,12816
6,www.icd9data.com,8693
7,www.wikidata.org,8069
8,amigo.geneontology.org,6909
9,www.nlm.nih.gov,6420


#### Top 20 hostnames found in W pages with external links

In [4]:
# count of top 20 hostnames in WP:M pages with external links
top_hosts_w_query = """
SELECT {} AS host, COUNT(DISTINCT el_from, el_to) AS count 
FROM ryanmax.population_externallinks
WHERE el_from IN 
    (SELECT DISTINCT page_id AS el_from
    FROM ryanmax.population_w_pages_with_extlinks
    )
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_w = spark.sql(
    top_hosts_w_query.format(
        parse_host.format('el_to')
    ))
top_hosts_w.toPandas()

,host,count
0,tools.wmflabs.org,2118863
1,www.google.com,1763467
2,doi.org,1199818
3,www.ncbi.nlm.nih.gov,1122007
4,www.worldcat.org,1061459
5,books.google.com,1231622
6,scholar.google.com,437109
7,www.wikidata.org,805309
8,www.jstor.org,548764
9,ssd.jpl.nasa.gov,592901


## Event Data

### all event types

#### Total event count for top 20 hostnames
- limited to WP:M pages with external links

In [5]:
# Total event count for top 20 hostnames
# limited to WP:M pages with external links
top_hosts_wpm_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_wpm_events = spark.sql(
    top_hosts_wpm_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_wpm_events.toPandas()

,host,total_events,daily_average_events
0,en.wikipedia.org,1860686,58146.43750
1,en.m.wikipedia.org,627309,19603.40625
2,www.ncbi.nlm.nih.gov,86432,2701.00000
3,doi.org,74930,2341.56250
4,books.google.com,19729,616.53125
5,www.drugs.com,16639,519.96875
6,www.who.int,15105,472.03125
7,translate.googleusercontent.com,10775,336.71875
8,theranos.com,10660,333.12500
9,www.cdc.gov,10119,316.21875


#### Total event count for top 20 hostnames
- limited to W pages with external links

In [6]:
# Total event count for top 20 hostnames by event type
# limited to W pages with external links
top_hosts_w_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_w_events = spark.sql(
    top_hosts_w_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_w_events.toPandas()

,host,total_events,daily_average_events
0,en.wikipedia.org,42334009,1.322938e+06
1,en.m.wikipedia.org,17636814,5.511504e+05
2,www.imdb.com,3050694,9.533419e+04
3,tools.wmflabs.org,1064153,3.325478e+04
4,www.youtube.com,601433,1.879478e+04
5,books.google.com,573246,1.791394e+04
6,www.espncricinfo.com,379919,1.187247e+04
7,doi.org,350714,1.095981e+04
8,www.nytimes.com,301568,9.424000e+03
9,www.bbc.co.uk,227540,7.110625e+03


### extClick events

#### Total event count for top 20 hostnames
- limited to WP:M pages with external links
- further limited to extClick events

In [7]:
# Total event count for top 20 hostnames
# limited to WP:M pages with external links
# further limited to extClick events
top_hosts_wpm_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action = 'extClick'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_wpm_events = spark.sql(
    top_hosts_wpm_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_wpm_events.toPandas()

,host,total_events,daily_average_events
0,www.ncbi.nlm.nih.gov,86432,2701.00000
1,doi.org,74930,2341.56250
2,books.google.com,19725,616.40625
3,www.drugs.com,16639,519.96875
4,www.who.int,15105,472.03125
5,theranos.com,10660,333.12500
6,www.cdc.gov,10119,316.21875
7,chemapps.stolaf.edu,7613,237.90625
8,apps.who.int,6384,199.50000
9,www.nlm.nih.gov,6291,196.59375


#### Total event count for top 20 hostnames by event type
- limited to W pages with external links
- further limited to extClick events

In [8]:
# Total event count for top 20 hostnames by event type
# limited to W pages with external links
# further limited to extClick events
top_hosts_w_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action = 'extClick'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_w_events = spark.sql(
    top_hosts_w_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_w_events.toPandas()

,host,total_events,daily_average_events
0,www.imdb.com,3050694,95334.18750
1,tools.wmflabs.org,1064153,33254.78125
2,www.youtube.com,601433,18794.78125
3,books.google.com,573062,17908.18750
4,www.espncricinfo.com,379919,11872.46875
5,doi.org,350714,10959.81250
6,www.nytimes.com,301568,9424.00000
7,www.bbc.co.uk,227540,7110.62500
8,www.theguardian.com,225815,7056.71875
9,www.instagram.com,182944,5717.00000


### limited to fnHover events

#### Total event count for top 20 hostnames
- limited to WP:M pages with external links
- further limited to fnHover events

In [9]:
# Total event count for top 20 hostnames
# limited to WP:M pages with external links
# further limited to fnHover events
top_hosts_wpm_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action = 'fnHover'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_wpm_events = spark.sql(
    top_hosts_wpm_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_wpm_events.toPandas()

,host,total_events,daily_average_events
0,en.wikipedia.org,1465057,45783.03125
1,en.m.wikipedia.org,89821,2806.90625
2,translate.googleusercontent.com,4032,126.00000
3,None,698,21.81250
4,www.translatoruser-int.com,149,4.65625
5,papago.naver.net,99,3.09375
6,www.biblegateway.com,39,1.21875
7,z5h64q92x9.net,21,0.65625
8,scholar.google.com,17,0.53125
9,emedien3.sub.uni-hamburg.de,12,0.37500


#### Total event count for top 20 hostnames by event type
- limited to W pages with external links
- further limited to fnHover events

In [10]:
# Total event count for top 20 hostnames by event type
# limited to W pages with external links
# further limited to fnHover events
top_hosts_w_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action = 'fnHover'
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_w_events = spark.sql(
    top_hosts_w_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_w_events.toPandas()

,host,total_events,daily_average_events
0,en.wikipedia.org,33809467,1.056546e+06
1,en.m.wikipedia.org,2015284,6.297762e+04
2,translate.googleusercontent.com,53714,1.678562e+03
3,None,27224,8.507500e+02
4,www.biblegateway.com,16145,5.045312e+02
5,www.translatoruser-int.com,2063,6.446875e+01
6,papago.naver.net,1250,3.906250e+01
7,z5h64q92x9.net,327,1.021875e+01
8,en.wikipedi0.org,246,7.687500e+00
9,emedien3.sub.uni-hamburg.de,169,5.281250e+00


### limited to extClick, fnClick OR upClick events

#### Total event count for top 20 hostnames
- limited to WP:M pages with external links
- further limited to extClick, fnClick OR upClick events

In [11]:
# Total event count for top 20 hostnames
# limited to WP:M pages with external links
# further limited to extClick, fnClick OR upClick events
top_hosts_wpm_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action in ('extClick', 'upClick','fnClick')
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_wpm_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_wpm_events = spark.sql(
    top_hosts_wpm_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_wpm_events.toPandas()

,host,total_events,daily_average_events
0,en.m.wikipedia.org,537488,16796.50000
1,en.wikipedia.org,395629,12363.40625
2,www.ncbi.nlm.nih.gov,86432,2701.00000
3,doi.org,74930,2341.56250
4,books.google.com,19728,616.50000
5,www.drugs.com,16639,519.96875
6,www.who.int,15105,472.03125
7,theranos.com,10660,333.12500
8,www.cdc.gov,10119,316.21875
9,chemapps.stolaf.edu,7613,237.90625


#### Total event count for top 20 hostnames by event type
- limited to W pages with external links
- further limited to extClick, fnClick OR upClick events

In [12]:
# Total event count for top 20 hostnames by event type
# limited to W pages with external links
# further limited to extClick, fnClick OR upClick events
top_hosts_w_query = """
SELECT {} AS host, COUNT(*) AS total_events, COUNT(*)/{} AS daily_average_events
FROM citationusage 
WHERE wiki = 'enwiki'
AND action in ('extClick', 'upClick','fnClick')
AND page_id IN 
    (SELECT DISTINCT page_id 
    FROM ryanmax.population_w_pages_with_extlinks
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_hosts_w_events = spark.sql(
    top_hosts_w_query.format(
        parse_host.format('link_url'),
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_hosts_w_events.toPandas()

,host,total_events,daily_average_events
0,en.m.wikipedia.org,15621530,488172.81250
1,en.wikipedia.org,8524542,266391.93750
2,www.imdb.com,3050694,95334.18750
3,tools.wmflabs.org,1064153,33254.78125
4,www.youtube.com,601433,18794.78125
5,books.google.com,573142,17910.68750
6,www.espncricinfo.com,379919,11872.46875
7,doi.org,350714,10959.81250
8,www.nytimes.com,301568,9424.00000
9,www.bbc.co.uk,227540,7110.62500


### Top wikipedia.org links by event type

In [13]:
# wikipedia.org links by event type
top_wiki_hosts_query = """
SELECT {} AS host, action, COUNT(*) AS count 
FROM citationusage 
WHERE wiki = 'enwiki'
AND lower(parse_url(link_url,'HOST')) like '%wikipedia.org'
AND (
        page_id IN 
        (SELECT page_id FROM ryanmax.population_w_pages_with_extlinks)
        OR page_id IN
        (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    )
{}
AND to_date(event_time) >= '{}'
AND to_date(event_time) <= '{}'
AND useragent_is_bot = FALSE
GROUP BY host, action
ORDER BY COUNT(*) DESC
LIMIT 20
"""
top_wiki_hosts_events = spark.sql(
    top_wiki_hosts_query.format(
        parse_host.format('link_url'),
        event_exclusion_sql, start_date_string, end_date_string
    ))
top_wiki_hosts_events_rdd = top_wiki_hosts_events.rdd
top_wiki_hosts_events_df = sqlContext.createDataFrame(top_wiki_hosts_events_rdd)
top_wiki_hosts_events_pandas = top_wiki_hosts_events_df.toPandas()

top_wiki_hosts_events_pandas.pivot(index='host', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
host,,,,
ar.wikipedia.org,276.0,NaN,NaN,NaN
de.wikipedia.org,1798.0,NaN,NaN,NaN
en.m.wikipedia.org,NaN,16044275.0,2105104.0,114695.0
en.wikipedia.org,262.0,7875877.0,35274475.0,1043977.0
eo.wikipedia.org,280.0,NaN,NaN,NaN
es.wikipedia.org,785.0,NaN,NaN,NaN
fr.wikipedia.org,1804.0,NaN,NaN,NaN
it.wikipedia.org,646.0,NaN,NaN,NaN
ja.wikipedia.org,1973.0,NaN,NaN,NaN
